# Tutoriel sur l'appel de fonctions parallèles avec Phi-4 Mini ONNX

Ce notebook montre comment utiliser Phi-4 Mini avec ONNX Runtime GenAI pour effectuer des appels de fonctions parallèles. L'appel de fonctions permet au modèle d'invoquer intelligemment des outils et des API externes en fonction des demandes de l'utilisateur.

## Aperçu

Dans ce tutoriel, vous apprendrez à :
- Configurer Phi-4 Mini avec ONNX Runtime GenAI
- Définir des schémas de fonctions pour réserver des vols et des hôtels
- Utiliser la génération guidée avec la grammaire Lark pour produire des résultats structurés
- Exécuter des appels de fonctions parallèles pour des scénarios complexes de réservation de voyage

## Prérequis

Avant de lancer ce notebook, assurez-vous d'avoir :
- Téléchargé le modèle Phi-4 Mini ONNX
- Installé le package `onnxruntime-genai`
- Une compréhension de base des concepts liés à l'appel de fonctions


## Étape 1 : Importer les bibliothèques nécessaires

Tout d'abord, nous allons importer les bibliothèques nécessaires pour l'implémentation de notre appel de fonction.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Étape 2 : Configuration et paramétrage du modèle

Nous allons maintenant configurer le modèle Phi-4 Mini ONNX. Assurez-vous de remplacer le chemin par le répertoire réel de votre modèle.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Étape 3 : Configurer les paramètres de génération

Configurez les paramètres de génération pour contrôler le comportement de sortie du modèle. Ces réglages garantissent des réponses déterministes et ciblées pour l'appel de fonctions.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Étape 4 : Définir les fonctions disponibles

Ici, nous définissons les fonctions que notre assistant IA peut utiliser. Dans cet exemple, nous avons deux fonctions liées aux voyages :
1. **booking_flight_tickets** : Pour réserver des vols entre aéroports
2. **booking_hotels** : Pour réserver des hébergements à l'hôtel

Les définitions des fonctions suivent le format de schéma d'appel de fonctions d'OpenAI.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Étape 5 : Fonctions auxiliaires pour la génération de grammaire

Ces fonctions auxiliaires transforment nos définitions de fonctions en format de grammaire Lark, utilisé par ONNX Runtime GenAI pour une génération guidée. Cela garantit que le modèle produit des appels de fonction valides dans le format JSON approprié.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Étape 6 : Tester la génération de grammaire

Testons nos fonctions de génération de grammaire pour voir comment elles convertissent nos définitions d'outils dans le format approprié.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Étape 7 : Préparer l'invite système et le générateur

Nous allons maintenant créer l'invite système qui informe le modèle des outils disponibles et configurer le générateur avec des paramètres de génération guidée.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Étape 8 : Initialiser le générateur avec une génération guidée

Nous allons maintenant créer le générateur avec nos paramètres configurés et appliquer la grammaire Lark pour une génération guidée.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Étape 9 : Tester l'appel de fonctions parallèles

Passons maintenant à la vérification de notre configuration avec un scénario complexe de réservation de voyage nécessitant l'appel de plusieurs fonctions.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Générer des appels de fonction

Le modèle va maintenant générer des appels de fonction structurés en fonction de la demande de l'utilisateur. Grâce à la génération guidée, la sortie sera au format JSON valide et pourra être exécutée directement.

**Résultat attendu** : Le modèle doit générer des appels de fonction pour :
1. `booking_flight_tickets` avec les détails de Pékin (PEK) à Paris (CDG)
2. `booking_hotels` avec les détails de l'hébergement à Paris

Exécutez la cellule ci-dessous pour voir la génération en direct :


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Conclusion

🎉 **Félicitations !** Vous avez réussi à implémenter l'appel de fonctions parallèles avec Phi-4 Mini en utilisant ONNX Runtime GenAI.

### Ce que vous avez appris :

1. **Configuration du modèle** : Comment configurer Phi-4 Mini avec ONNX Runtime GenAI
2. **Définition des fonctions** : Comment définir des schémas de fonctions pour l'appel de fonctions IA
3. **Génération guidée** : Comment utiliser la grammaire Lark pour produire des sorties structurées
4. **Appels de fonctions parallèles** : Comment gérer des requêtes complexes nécessitant plusieurs appels de fonctions

### Principaux avantages :

- ✅ **Sortie structurée** : La génération guidée garantit des appels de fonctions JSON valides
- ✅ **Traitement parallèle** : Gérer plusieurs appels de fonctions dans une seule requête
- ✅ **Haute performance** : ONNX Runtime offre une inférence optimisée
- ✅ **Schéma flexible** : Facile d'ajouter ou de modifier des définitions de fonctions

### Ressources :

- [Documentation de Phi-4 Mini](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [Documentation ONNX Runtime GenAI](https://onnxruntime.ai/docs/genai/)
- [Bonnes pratiques pour l'appel de fonctions](https://platform.openai.com/docs/guides/function-calling)



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
